In [9]:
from datetime import datetime
import glob

import pandas as pd
import numpy as np

from fasttext_worker import pipeline, LABEL_COLUMNS

In [2]:
args ={}

args["train_path"] = "input/train_small.csv"
args["test_path"] = "input/test_small.csv"
args["train_path"] = "input/train.csv.zip"
args["test_path"] = "input/test.csv.zip"
args["n_splits"] = 10

args["save_proba"] = "probas/"

In [5]:
train = pd.read_csv(args["train_path"])
test = pd.read_csv(args["test_path"])

def get_features(path):
    data = pd.read_pickle(path)
    columns = [col for col in list(data) if "_proba_" in col]
    return data[columns].as_matrix()

train_paths = [path for path in glob.glob(args["save_proba"] + "*") if "train" in path]
test_paths = [path for path in glob.glob(args["save_proba"] + "*") if "test" in path]

train_features = np.hstack([get_features(path) for path in train_paths])
train_labels = train[LABEL_COLUMNS].as_matrix()

test_features = np.hstack([get_features(path) for path in test_paths])

In [30]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


def fix_pred(probas):
    return np.hstack(proba[:, 1][:, np.newaxis] for proba in probas)


kfold = KFold(n_splits=args["n_splits"])
roc_aucs = []
train_pred = np.zeros(train_labels.shape)
test_pred = np.zeros((test_features.shape[0], train_labels.shape[1]))
for i, (train_index, val_index) in enumerate(kfold.split(train)):
    print("Fold %s in progress..." % i)
    train_features_ = train_features[train_index, :]
    train_labels_ = train_labels[train_index, :]
    val_features_ = train_features[val_index, :]
    val_labels_ = train_labels[val_index, :]
    
    from catboost import CatBoostClassifier
    for i in range(len(LABEL_COLUMNS)):
        model = CatBoostClassifier(loss_function='Logloss', logging_level='Verbose',
                                   iterations)
    #     model = ExtraTreesClassifier(min_samples_leaf=50, n_jobs=-1)
        model.fit(train_features_, train_labels_[:, i])
        print("train roc_auc:", LABEL_COLUMNS[i],
              roc_auc_score(train_labels_[:, i], model.predict_proba(train_features_)))
        print("val roc_auc:", LABEL_COLUMNS[i],
              roc_auc_score(val_labels_[:, i], model.predict_proba(val_features_)))

        train_pred[:, i] += model.predict_proba(train_features)
        test_pred[:, i] += model.predict_proba(test_features)
    print("Fold %s completed" % i)

train_pred = train_pred / args["n_splits"]
test_pred = test_pred / args["n_splits"]


Fold 0 in progress...
0:	learn: 0.6062579	total: 121ms	remaining: 9.55s
1:	learn: 0.5485793	total: 244ms	remaining: 9.53s
2:	learn: 0.4975906	total: 384ms	remaining: 9.86s
3:	learn: 0.4540137	total: 511ms	remaining: 9.71s
4:	learn: 0.4093827	total: 655ms	remaining: 9.83s
5:	learn: 0.3674007	total: 848ms	remaining: 10.5s
6:	learn: 0.3307088	total: 1.15s	remaining: 12s
7:	learn: 0.3064244	total: 1.39s	remaining: 12.5s
8:	learn: 0.2854983	total: 1.56s	remaining: 12.3s
9:	learn: 0.2633559	total: 1.7s	remaining: 11.9s
10:	learn: 0.2436962	total: 1.81s	remaining: 11.4s
11:	learn: 0.2245191	total: 1.95s	remaining: 11s
12:	learn: 0.2071420	total: 2.07s	remaining: 10.7s
13:	learn: 0.1930027	total: 2.21s	remaining: 10.4s
14:	learn: 0.1807335	total: 2.34s	remaining: 10.1s
15:	learn: 0.1704213	total: 2.48s	remaining: 9.93s
16:	learn: 0.1605655	total: 2.6s	remaining: 9.65s
17:	learn: 0.1501597	total: 2.76s	remaining: 9.5s
18:	learn: 0.1420498	total: 2.88s	remaining: 9.26s
19:	learn: 0.1352267	total

val roc_auc: severe_toxic 0.989828845699
0:	learn: 0.5872183	total: 251ms	remaining: 19.9s
1:	learn: 0.5089442	total: 368ms	remaining: 14.4s
2:	learn: 0.4346369	total: 499ms	remaining: 12.8s
3:	learn: 0.3738533	total: 617ms	remaining: 11.7s
4:	learn: 0.3232462	total: 757ms	remaining: 11.4s
5:	learn: 0.2918064	total: 898ms	remaining: 11.1s
6:	learn: 0.2514227	total: 1.04s	remaining: 10.9s
7:	learn: 0.2207865	total: 1.18s	remaining: 10.6s
8:	learn: 0.1952259	total: 1.48s	remaining: 11.7s
9:	learn: 0.1737097	total: 1.86s	remaining: 13s
10:	learn: 0.1544914	total: 2.06s	remaining: 12.9s
11:	learn: 0.1389755	total: 2.22s	remaining: 12.6s
12:	learn: 0.1255872	total: 2.36s	remaining: 12.2s
13:	learn: 0.1129335	total: 2.5s	remaining: 11.8s
14:	learn: 0.1043395	total: 2.64s	remaining: 11.4s
15:	learn: 0.0949708	total: 2.78s	remaining: 11.1s
16:	learn: 0.0882185	total: 2.9s	remaining: 10.8s
17:	learn: 0.0830139	total: 3.06s	remaining: 10.5s
18:	learn: 0.0781142	total: 3.19s	remaining: 10.2s
19:	

train roc_auc: threat 0.989415078067
val roc_auc: threat 0.985098880788
0:	learn: 0.6181472	total: 113ms	remaining: 8.96s
1:	learn: 0.5196754	total: 234ms	remaining: 9.14s
2:	learn: 0.4464898	total: 366ms	remaining: 9.4s
3:	learn: 0.3950884	total: 487ms	remaining: 9.26s
4:	learn: 0.3552741	total: 621ms	remaining: 9.32s
5:	learn: 0.3177875	total: 737ms	remaining: 9.09s
6:	learn: 0.2777308	total: 868ms	remaining: 9.05s
7:	learn: 0.2555782	total: 990ms	remaining: 8.91s
8:	learn: 0.2335995	total: 1.12s	remaining: 8.87s
9:	learn: 0.2102104	total: 1.24s	remaining: 8.67s
10:	learn: 0.1884865	total: 1.37s	remaining: 8.57s
11:	learn: 0.1717747	total: 1.49s	remaining: 8.43s
12:	learn: 0.1572594	total: 1.62s	remaining: 8.34s
13:	learn: 0.1457297	total: 1.74s	remaining: 8.22s
14:	learn: 0.1361310	total: 1.87s	remaining: 8.12s
15:	learn: 0.1286228	total: 1.99s	remaining: 7.97s
16:	learn: 0.1232558	total: 2.13s	remaining: 7.91s
17:	learn: 0.1176847	total: 2.27s	remaining: 7.83s
18:	learn: 0.1119253	

train roc_auc: identity_hate 0.988615260018
val roc_auc: identity_hate 0.981231082556
Fold 0 completed, val roc_auc 0.988086056682
Fold 1 in progress...
0:	learn: 0.6333197	total: 115ms	remaining: 9.11s
1:	learn: 0.5767068	total: 230ms	remaining: 8.96s
2:	learn: 0.5206200	total: 361ms	remaining: 9.27s
3:	learn: 0.4744301	total: 483ms	remaining: 9.18s
4:	learn: 0.4247850	total: 626ms	remaining: 9.38s
5:	learn: 0.3898618	total: 740ms	remaining: 9.12s
6:	learn: 0.3473522	total: 873ms	remaining: 9.1s
7:	learn: 0.3145409	total: 995ms	remaining: 8.95s
8:	learn: 0.2897162	total: 1.13s	remaining: 8.91s
9:	learn: 0.2604684	total: 1.25s	remaining: 8.77s
10:	learn: 0.2370862	total: 1.38s	remaining: 8.64s
11:	learn: 0.2181163	total: 1.5s	remaining: 8.49s
12:	learn: 0.2031346	total: 1.62s	remaining: 8.37s
13:	learn: 0.1923082	total: 1.74s	remaining: 8.19s
14:	learn: 0.1734761	total: 1.86s	remaining: 8.07s
15:	learn: 0.1626383	total: 1.98s	remaining: 7.93s
16:	learn: 0.1538070	total: 2.11s	remaining

78:	learn: 0.0194809	total: 16.9s	remaining: 214ms
79:	learn: 0.0193878	total: 17.2s	remaining: 0us
train roc_auc: severe_toxic 0.994333193091
val roc_auc: severe_toxic 0.990177059338
0:	learn: 0.6055586	total: 120ms	remaining: 9.46s
1:	learn: 0.5352940	total: 238ms	remaining: 9.3s
2:	learn: 0.4629950	total: 383ms	remaining: 9.84s
3:	learn: 0.4171953	total: 519ms	remaining: 9.86s
4:	learn: 0.3710437	total: 665ms	remaining: 9.98s
5:	learn: 0.3283865	total: 804ms	remaining: 9.92s
6:	learn: 0.2949102	total: 943ms	remaining: 9.83s
7:	learn: 0.2639317	total: 1.08s	remaining: 9.71s
8:	learn: 0.2329208	total: 1.22s	remaining: 9.63s
9:	learn: 0.2024143	total: 1.37s	remaining: 9.61s
10:	learn: 0.1897246	total: 1.53s	remaining: 9.58s
11:	learn: 0.1710363	total: 1.66s	remaining: 9.4s
12:	learn: 0.1550826	total: 1.8s	remaining: 9.28s
13:	learn: 0.1443546	total: 1.94s	remaining: 9.13s
14:	learn: 0.1316110	total: 2.09s	remaining: 9.05s
15:	learn: 0.1220091	total: 2.23s	remaining: 8.9s
16:	learn: 0.1

...

62:	learn: 0.0061606	total: 8.56s	remaining: 2.31s
63:	learn: 0.0060982	total: 8.69s	remaining: 2.17s
64:	learn: 0.0060332	total: 8.83s	remaining: 2.04s
65:	learn: 0.0059745	total: 8.97s	remaining: 1.9s
66:	learn: 0.0059365	total: 9.11s	remaining: 1.77s
67:	learn: 0.0058924	total: 9.25s	remaining: 1.63s
68:	learn: 0.0058404	total: 9.39s	remaining: 1.5s
69:	learn: 0.0058026	total: 9.52s	remaining: 1.36s
70:	learn: 0.0057488	total: 9.66s	remaining: 1.22s
71:	learn: 0.0057044	total: 9.79s	remaining: 1.09s
72:	learn: 0.0056529	total: 9.93s	remaining: 952ms
73:	learn: 0.0056210	total: 10.1s	remaining: 816ms
74:	learn: 0.0055892	total: 10.2s	remaining: 680ms
75:	learn: 0.0055391	total: 10.3s	remaining: 544ms
76:	learn: 0.0054956	total: 10.5s	remaining: 409ms
77:	learn: 0.0054477	total: 10.6s	remaining: 272ms
78:	learn: 0.0054144	total: 10.8s	remaining: 136ms
79:	learn: 0.0053728	total: 10.9s	remaining: 0us
train roc_auc: threat 0.992442738955
val roc_auc: threat 0.981112438665
0:	learn: 0.60

62:	learn: 0.0219960	total: 8.72s	remaining: 2.35s
63:	learn: 0.0218705	total: 8.86s	remaining: 2.21s
64:	learn: 0.0217631	total: 9s	remaining: 2.08s
65:	learn: 0.0216506	total: 9.13s	remaining: 1.94s
66:	learn: 0.0215837	total: 9.28s	remaining: 1.8s
67:	learn: 0.0214450	total: 9.41s	remaining: 1.66s
68:	learn: 0.0213409	total: 9.56s	remaining: 1.52s
69:	learn: 0.0212128	total: 9.7s	remaining: 1.39s
70:	learn: 0.0211547	total: 9.85s	remaining: 1.25s
71:	learn: 0.0209699	total: 9.98s	remaining: 1.11s
72:	learn: 0.0208755	total: 10.1s	remaining: 972ms
73:	learn: 0.0207468	total: 10.3s	remaining: 833ms
74:	learn: 0.0206807	total: 10.4s	remaining: 695ms
75:	learn: 0.0206282	total: 10.6s	remaining: 556ms
76:	learn: 0.0204669	total: 10.7s	remaining: 418ms
77:	learn: 0.0204190	total: 10.9s	remaining: 278ms
78:	learn: 0.0203745	total: 11s	remaining: 139ms
79:	learn: 0.0202954	total: 11.1s	remaining: 0us
train roc_auc: identity_hate 0.992204371262
val roc_auc: identity_hate 0.982343578177
Fold 

60:	learn: 0.0208684	total: 8.46s	remaining: 2.63s
61:	learn: 0.0208093	total: 8.6s	remaining: 2.5s
62:	learn: 0.0207344	total: 8.74s	remaining: 2.36s
63:	learn: 0.0206821	total: 8.87s	remaining: 2.22s
64:	learn: 0.0206395	total: 9.02s	remaining: 2.08s
65:	learn: 0.0205721	total: 9.15s	remaining: 1.94s
66:	learn: 0.0204947	total: 9.3s	remaining: 1.8s
67:	learn: 0.0204092	total: 9.45s	remaining: 1.67s
68:	learn: 0.0203659	total: 9.6s	remaining: 1.53s
69:	learn: 0.0202818	total: 9.73s	remaining: 1.39s
70:	learn: 0.0202450	total: 9.87s	remaining: 1.25s
71:	learn: 0.0202049	total: 10s	remaining: 1.11s
72:	learn: 0.0201634	total: 10.2s	remaining: 973ms
73:	learn: 0.0201434	total: 10.3s	remaining: 834ms
74:	learn: 0.0200900	total: 10.4s	remaining: 695ms
75:	learn: 0.0200225	total: 10.6s	remaining: 556ms
76:	learn: 0.0199494	total: 10.7s	remaining: 417ms
77:	learn: 0.0198968	total: 10.8s	remaining: 278ms
78:	learn: 0.0198264	total: 11s	remaining: 139ms
79:	learn: 0.0197808	total: 11.1s	remain

60:	learn: 0.0084647	total: 8.44s	remaining: 2.63s
61:	learn: 0.0083849	total: 8.57s	remaining: 2.49s
62:	learn: 0.0083353	total: 8.72s	remaining: 2.35s
63:	learn: 0.0082732	total: 8.85s	remaining: 2.21s
64:	learn: 0.0082057	total: 9.01s	remaining: 2.08s
65:	learn: 0.0081403	total: 9.14s	remaining: 1.94s
66:	learn: 0.0080419	total: 9.29s	remaining: 1.8s
67:	learn: 0.0079746	total: 9.43s	remaining: 1.66s
68:	learn: 0.0078631	total: 9.57s	remaining: 1.52s
69:	learn: 0.0078065	total: 9.71s	remaining: 1.39s
70:	learn: 0.0077751	total: 9.86s	remaining: 1.25s
71:	learn: 0.0077363	total: 10s	remaining: 1.11s
72:	learn: 0.0076682	total: 10.1s	remaining: 973ms
73:	learn: 0.0076245	total: 10.3s	remaining: 834ms
74:	learn: 0.0075852	total: 10.4s	remaining: 695ms
75:	learn: 0.0075521	total: 10.6s	remaining: 556ms
76:	learn: 0.0074730	total: 10.7s	remaining: 417ms
77:	learn: 0.0074047	total: 10.8s	remaining: 278ms
78:	learn: 0.0073497	total: 11s	remaining: 139ms
79:	learn: 0.0073004	total: 11.1s	re

60:	learn: 0.0202182	total: 8.48s	remaining: 2.64s
61:	learn: 0.0200409	total: 8.61s	remaining: 2.5s
62:	learn: 0.0198925	total: 8.78s	remaining: 2.37s
63:	learn: 0.0197888	total: 8.91s	remaining: 2.23s
64:	learn: 0.0196166	total: 9.05s	remaining: 2.09s
65:	learn: 0.0194873	total: 9.19s	remaining: 1.95s
66:	learn: 0.0193695	total: 9.34s	remaining: 1.81s
67:	learn: 0.0192727	total: 9.47s	remaining: 1.67s
68:	learn: 0.0191443	total: 9.63s	remaining: 1.53s
69:	learn: 0.0190652	total: 9.76s	remaining: 1.39s
70:	learn: 0.0189669	total: 9.9s	remaining: 1.25s
71:	learn: 0.0188518	total: 10s	remaining: 1.11s
72:	learn: 0.0187494	total: 10.2s	remaining: 977ms
73:	learn: 0.0186609	total: 10.3s	remaining: 837ms
74:	learn: 0.0185461	total: 10.5s	remaining: 698ms
75:	learn: 0.0184545	total: 10.6s	remaining: 559ms
76:	learn: 0.0183473	total: 10.8s	remaining: 419ms
77:	learn: 0.0182522	total: 10.9s	remaining: 280ms
78:	learn: 0.0181875	total: 11s	remaining: 140ms
79:	learn: 0.0180693	total: 11.2s	rem

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


def fix_pred(probas):
    return np.hstack(proba[:, 1][:, np.newaxis] for proba in probas)


kfold = KFold(n_splits=args["n_splits"])
roc_aucs = []
train_pred = np.zeros(train_labels.shape)
test_pred = np.zeros((test_features.shape[0], train_labels.shape[1]))
for i, (val_index, train_index) in enumerate(kfold.split(train)):
    print("Fold %s in progress..." % i)
    train_features_ = train_features[train_index, :]
    train_labels_ = train_labels[train_index, :]
    val_features_ = train_features[val_index, :]
    val_labels_ = train_labels[val_index, :]
    
    from catboost import CatBoostClassifier
    val_roc_auc = []
    for j in range(len(LABEL_COLUMNS)):
        model = CatBoostClassifier(loss_function='Logloss', iterations=80)
    #     model = ExtraTreesClassifier(min_samples_leaf=50, n_jobs=-1)
        model.fit(train_features_, train_labels_[:, j])
        train_pred_ = model.predict_proba(train_features_)[:, 1]
        val_pred_ = model.predict_proba(val_features_)[:, 1]
        print("train roc_auc:", LABEL_COLUMNS[j], roc_auc_score(train_labels_[:, j], train_pred_))
        val_roc_auc.append(roc_auc_score(val_labels_[:, j], val_pred_))
        print("val roc_auc:", LABEL_COLUMNS[j], val_roc_auc[-1])

        test_pred[:, j] += model.predict_proba(test_features)[:, 1]
    print("Fold %s completed, val roc_auc %s" % (i, np.mean(val_roc_auc) ))

train_pred = train_pred / args["n_splits"]
test_pred = test_pred / args["n_splits"]


In [31]:
for i, label in enumerate(LABEL_COLUMNS):
    test[label] = test_pred[:, i]

output = "output/%s-fasttext-baseline-submission.txt" % datetime.now()
cols = ["id"] + LABEL_COLUMNS
test[cols].to_csv(output, sep=",", index=False, quotechar=" ", float_format="%.16f")